In [1]:
import numpy as np
import torch
import torch.optim as optim
import pickle
import pandas as pd
import sys
import copy

sys.path.append("..\\..\\src")

from datasets import CV2ImageDataset, dataset_loader
from tf_model import Net
from tqdm import tqdm
from model_class import NeuralNet
from run_phase import run_phase
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import StratifiedKFold
import pickle
from meta_learning_tools import MetaLearn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
def model_prototype():
    '''
    Creates a Neural Network 
    Returns:
        net:  Neural Network
    '''
    net = Net()
    nn_model = NeuralNet(net)
    net = nn_model.get_model()
    return net

In [3]:
with open('saved_cifar_model_weights_dictionary_with_tl.pkl', 'rb') as fp:
    model_weight_dict = pickle.load(fp)

In [4]:
model_list = []
for key in tqdm(model_weight_dict.keys()):
    model_weight = model_weight_dict[key]
    model = model_prototype()
    model.load_state_dict(model_weight)
    model.cpu()
    model.eval()
    model_list.append(model)

100%|████████████████████████████████████████████████████████████| 5/5 [00:13<00:00,  2.74s/it]


In [13]:
train_df = pd.read_csv('data\\train.csv')
val_df = pd.read_csv('data\\test.csv')



print('train_df initial shape: ',train_df.shape)
print('val_df intial shape: ', val_df.shape)

train_df = train_df[0:10000]
val_df = val_df[0:7500]


print('train_df shape: ',train_df.shape)
print('val_df shape: ', val_df.shape)

train_df initial shape:  (50000, 5)
val_df intial shape:  (10000, 5)
train_df shape:  (10000, 5)
val_df shape:  (7500, 5)


In [14]:
meta_learn = MetaLearn(model_list)
tta_transforms = [
    A.augmentations.transforms.ChannelShuffle(p=0.5),
    A.HorizontalFlip(p=0.5), 
]

print('Generate Dictionary of Metalearning Dataframes')
train_tta_dict= meta_learn.generate_tta_dict_for_folds(train_df , tta_transforms, device)
val_tta_dict= meta_learn.generate_tta_dict_for_folds(val_df , tta_transforms, device)

Generate Dictionary of Metalearning Dataframes


  0%|                                                                | 0/10000 [00:00<?, ?it/s]E:\anaconda3\envs\pytorch\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
100%|██████████████████████████████████████████████████████| 7500/7500 [24:16<00:00,  5.15it/s]


In [15]:
with open('train_tta_dict.pkl', 'wb') as fp:
    pickle.dump(train_tta_dict, fp)
with open('val_tta_dict.pkl', 'wb') as fp:
    pickle.dump(val_tta_dict, fp)

In [16]:
print(val_tta_dict.keys())

dict_keys([0, 1, 2, 3, 4])


In [17]:
from datasets import CV2ImageDataset, dataset_loader
from run_phase import run_phase
batch_size = 64
aug = A.Compose([ 
    A.Normalize(),            
    ToTensorV2()])
val_ds = CV2ImageDataset(val_df, transform=aug, device = device)
val_ds_l = dataset_loader(val_ds, batch_size = batch_size)
valloader = val_ds_l.get_dataloader()
#Sanity Check
for i, model in enumerate(model_list):
    print('model : ', i)
    val_phase = run_phase(valloader, model.to(device),'Validation', device, loss = torch.nn.CrossEntropyLoss( ))
    val_accuracy_meter, val_loss_meter =  val_phase.run()
    model.cpu()
    model.eval()
    

model :  0


Validation: 100%|████████████████████████████████████████████| 118/118 [00:15<00:00,  7.40it/s]


Validation Accuracy for epoch : 0.8333333333333334
Validation Loss for epoch : 0.007860651195049286
model :  1


Validation: 100%|████████████████████████████████████████████| 118/118 [00:15<00:00,  7.56it/s]


Validation Accuracy for epoch : 0.8370666666666666
Validation Loss for epoch : 0.0075832400500774385
model :  2


...

Validation Accuracy for epoch : 0.8381333333333333
Validation Loss for epoch : 0.007580565561850865
model :  3


Validation: 100%|████████████████████████████████████████████| 118/118 [00:15<00:00,  7.43it/s]


Validation Accuracy for epoch : 0.8410666666666666
Validation Loss for epoch : 0.007529564102490743
model :  4


Validation: 100%|████████████████████████████████████████████| 118/118 [00:15<00:00,  7.48it/s]


Validation Accuracy for epoch : 0.8408
Validation Loss for epoch : 0.0075529712657133735


In [18]:
train_tta_df=meta_learn.create_meta_learn_labels_on_dict(train_tta_dict)
val_tta_df=meta_learn.create_meta_learn_labels_on_dict(val_tta_dict)
print( val_tta_df.shape,train_tta_df.shape)

5it [00:00, 624.91it/s]
5it [00:00, 992.55it/s]

(7500, 32) (10000, 32)


In [19]:
val_tta_df.head()

,Unnamed: 0,filenames,label_names,path,label,tta_0_custom_0,0_custom_0_aug_0,0_custom_0_aug_1,Unnamed: 0,filenames,...,label_names,tta_3_custom_3,3_custom_3_aug_0,3_custom_3_aug_1,Unnamed: 0,filenames,label_names,tta_4_custom_4,4_custom_4_aug_0,4_custom_4_aug_1
0,0,domestic_cat_s_000907.png,cat,E:\Git\Writing Library\Classification\Examples...,3,3,3,3,0,domestic_cat_s_000907.png,...,cat,3,3,3,0,domestic_cat_s_000907.png,cat,3,3,3
1,1,hydrofoil_s_000078.png,ship,E:\Git\Writing Library\Classification\Examples...,8,8,8,8,1,hydrofoil_s_000078.png,...,ship,8,8,8,1,hydrofoil_s_000078.png,ship,8,8,8
2,2,sea_boat_s_001456.png,ship,E:\Git\Writing Library\Classification\Examples...,8,8,8,8,2,sea_boat_s_001456.png,...,ship,8,8,8,2,sea_boat_s_001456.png,ship,8,8,8
3,3,jetliner_s_001705.png,airplane,E:\Git\Writing Library\Classification\Examples...,0,0,0,0,3,jetliner_s_001705.png,...,airplane,0,0,0,3,jetliner_s_001705.png,airplane,0,0,0
4,4,green_frog_s_001658.png,frog,E:\Git\Writing Library\Classification\Examples...,6,6,8,6,4,green_frog_s_001658.png,...,frog,6,8,6,4,green_frog_s_001658.png,frog,6,8,6


In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

val_acc, val_log_loss = meta_learn.do_meta_learning_on_tta_dicts(RandomForestClassifier(), train_tta_df, val_tta_df, 'custom')

print('RandomForestClassifier Validation Accuracy, Validation Loss: ',val_acc, val_log_loss)

val_acc, val_log_loss = meta_learn.do_meta_learning_on_tta_dicts(DecisionTreeClassifier(), train_tta_df, val_tta_df, 'custom')

print('DecisionTreeClassifier Validation Accuracy, Validation Loss: ',val_acc, val_log_loss)

val_acc, val_log_loss = meta_learn.do_meta_learning_on_tta_dicts(GradientBoostingClassifier(), train_tta_df, val_tta_df, 'custom')

print('GradientBoostingClassifier Validation Accuracy, Validation Loss: ',val_acc, val_log_loss)

E:\Git\Writing Library\Classification\Examples\Cifar\..\..\src\meta_learning_tools.py:141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(train_data, train_labels)


RandomForestClassifier Validation Accuracy, Validation Loss:  0.8405333333333334 1.3168286186822704
DecisionTreeClassifier Validation Accuracy, Validation Loss:  0.8405333333333334 1.5398482400342914


E:\anaconda3\envs\pytorch\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier Validation Accuracy, Validation Loss:  0.8402666666666667 0.751826037518652
